In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0,1,2,3'

In [2]:
%run -p ../train_cnf_disentangle_rl_2cond_beta.py --data colormnist --dims 64,64,64 --strides 1,1,1,1 --num_blocks 2 --layer_type concat --multiscale True --rademacher True --batch_size 900 --test_batch_size 500 --save ../experiments_published/infocnf_disentangle_colormnist_bs900_sratio_1_4th_drop_0_5_rl_stdscale_6_2cond_linear_beta_m0_01_run1 --seed 1 --lr 0.001 --conditional True --controlled_tol False --train_mode semisup --log_freq 10 --weight_y 0.5 --condition_ratio 0.25 --dropout_rate 0.5 --scale_fac 1.0 --scale_std 6.0 --cond_nn linear --y_color 10 --y_class 10 --beta -0.01
#

/tancode/repos/tan-ffjord/train_cnf_disentangle_rl_2cond.py
from __future__ import print_function

import argparse
import os
import time
import numpy as np

import torch
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as tforms
from torchvision.utils import save_image

import torch.utils.data as data
from torch.utils.data import Dataset

from PIL import Image
import os.path
import errno
import codecs

import lib.layers as layers
import lib.utils as utils
import lib.multiscale_parallel as multiscale_parallel
import lib.modules as modules
import lib.thops as thops

from train_misc import standard_normal_logprob
from train_misc import set_cnf_options, count_nfe, count_parameters, count_total_time, count_nfe_gate
from train_misc import add_spectral_norm, spectral_norm_power_iteration
from train_misc import create_regularization_fns, get_regularization, append_regularization_to_log

from tensorboardX import SummaryWriter

# go fast boi!!
torch.b

AssertionError: non-finite values in state `y`: tensor([ 0.0118,  0.0123,  0.0124,  ..., -1.1926, -0.1269, -0.0779],
       device='cuda:3')

         549525800 function calls (513432293 primitive calls) in 35690.634 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     3138 21163.401    6.744 21163.401    6.744 {method 'run_backward' of 'torch._C._EngineBase' objects}
   138588 12486.118    0.090 12486.118    0.090 {method 'acquire' of '_thread.lock' objects}
   108324  199.223    0.002  199.223    0.002 {method '_write_file' of 'torch._C.CudaFloatStorageBase' objects}
  3294200  196.581    0.000 1182.447    0.000 train_cnf_disentangle_rl_2cond.py:205(__getitem__)
  3294200  147.163    0.000  236.931    0.000 train_cnf_disentangle_rl_2cond.py:308(add_noise)
  3294200   87.506    0.000  380.506    0.000 functional.py:32(to_tensor)
 17816594   69.978    0.000  108.392    0.000 module.py:537(__setattr__)
  3294348   68.224    0.000   68.224    0.000 {method 'uniform_' of 'torch._C._TensorBase' objects}
  3294200   65.707    0.000   65.707    0.000 {method 'div' of '